In [0]:
# importing all needed libraries
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import geopy
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# loading our dataframe from the last saved file
df_main = pd.read_csv("final_dataset_cord.csv", index_col=0)
df_main.head()

,postal_code,borough,neighbourhood,lati,long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


Creating a dataframe contains only boroughs that contain the word tornto in them

In [3]:
df_tor = df_main[df_main.borough.str.contains("Toronto")]
df_tor.head()

,postal_code,borough,neighbourhood,lati,long
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [4]:
df_tor.shape

(39, 5)

In [0]:
CLIENT_ID = "GY1OE4RM5PUSIPBLIRX4HZOSZAZIZP0WXLEYG54MOCFKRYNL"
CLIENT_SECRET = "Z0DS1ERJRGSY1P2FAETOORK3UZ3I3RBUL0NB1TS2VUCD3PYU"
VERSION = '20180605'

Creating a function to get all the venues of our neighbourhoods

In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500, LIMIT = 100):
    """This function gets all venues for a given neigbourhoods and return a dataframe with the resulting venues"""
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Venue Category ID']
    
    return(nearby_venues)

Calling the above function on each neighborhood and create **toronto_venues** dataframe

In [7]:
toronto_venues = getNearbyVenues(names=df_tor['neighbourhood'],
                                   latitudes=df_tor['lati'],
                                   longitudes=df_tor['long'])


Harbourfront
Queen's Park
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, King, Adelaide
Dufferin, Dovercourt Village
Union Station, Toronto Islands, Harbourfront East
Trinity, Little Portugal
Riverdale, The Danforth West
Toronto Dominion Centre, Design Exchange
Parkdale Village, Exhibition Place, Brockton
India Bazaar, The Beaches West
Victoria Hotel, Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill West, Forest Hill North
The Junction South, High Park
North Toronto West
Yorkville, North Midtown, The Annex
Roncesvalles, Parkdale
Davisville
University of Toronto, Harbord
Swansea, Runnymede
Summerhill East, Moore Park
Kensington Market, Grange Park, Chinatown
Summerhill West, South Hill, Rathnelly, Forest Hill SE, Deer Park
South Niagara, Railway Lands, King and Spadina, Harbourfront West, Island airport, Bathurst Quay, CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
St. James Town, Cabbagetown
Und

 Exploring the resulting dataframe

In [8]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Category ID
0,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery,4bf58dd8d48988d16a941735
1,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop,4bf58dd8d48988d1e0931735
2,Harbourfront,43.654260,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center,4bf58dd8d48988d175941735
3,Harbourfront,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant,4bf58dd8d48988d1c4941735
4,Harbourfront,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site,4deefb944765f83613cdba6e
...,...,...,...,...,...,...,...,...
3750,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Fairmount Park,43.675286,-79.315745,Baseball Field,4bf58dd8d48988d1e8941735
3751,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Sushi Marche,43.662033,-79.335626,Sushi Restaurant,4bf58dd8d48988d1d2941735
3752,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Remarkable Bean Leslieville,43.662010,-79.335709,Café,4bf58dd8d48988d16d941735
3753,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,New Town Restaurant,43.672880,-79.319488,Diner,4bf58dd8d48988d147941735


The venues of each neighborhood

In [0]:
list_of_catgories_id = ['4d4b7104d754a06370d81259', '56aa371be4b08b9a8d5734db', '4fceea171983d5d06c3e9823', '4bf58dd8d48988d1e1931735', '4bf58dd8d48988d1e2931735', '4bf58dd8d48988d1e4931735', '4bf58dd8d48988d17c941735', '52e81612bcbc57f1066b79e7', '4bf58dd8d48988d18e941735', '5032792091d4c4b30a586d5c', '52e81612bcbc57f1066b79ef', '52e81612bcbc57f1066b79e8', '56aa371be4b08b9a8d573532', '4bf58dd8d48988d1f1931735', '52e81612bcbc57f1066b79ea', '4deefb944765f83613cdba6e', '5744ccdfe4b0c0459246b4bb', '52e81612bcbc57f1066b79e6', '5642206c498e4bfca532186c', '52e81612bcbc57f1066b79eb', '4bf58dd8d48988d17f941735', '56aa371be4b08b9a8d5734de', '4bf58dd8d48988d17e941735', '4bf58dd8d48988d180941735', '4bf58dd8d48988d181941735', '4bf58dd8d48988d18f941735', '559acbe0498e472f1a53fa23', '4bf58dd8d48988d190941735', '4bf58dd8d48988d192941735', '4bf58dd8d48988d191941735', '4bf58dd8d48988d1e5931735', '4bf58dd8d48988d1e7931735', '4bf58dd8d48988d1e8931735', '4bf58dd8d48988d1e9931735', '5744ccdfe4b0c0459246b4b8', '4bf58dd8d48988d1f2931735', '4bf58dd8d48988d134941735', '4bf58dd8d48988d135941735', '4bf58dd8d48988d136941735', '4bf58dd8d48988d137941735', '4bf58dd8d48988d1e3931735', '507c8c4091d498d9fc8c67a9', '52e81612bcbc57f1066b79ed', '52e81612bcbc57f1066b79ee', '56aa371be4b08b9a8d573514', '4bf58dd8d48988d1f4931735', '52e81612bcbc57f1066b79e9', '52e81612bcbc57f1066b79ec', '56aa371be4b08b9a8d5734f9', '4bf58dd8d48988d184941735', '4bf58dd8d48988d18c941735', '4bf58dd8d48988d18b941735', '4bf58dd8d48988d18a941735', '4bf58dd8d48988d189941735', '4bf58dd8d48988d185941735', '56aa371be4b08b9a8d573556', '4bf58dd8d48988d188941735', '4e39a891bd410d7aed40cbc2', '4bf58dd8d48988d187941735', '4bf58dd8d48988d182941735', '5109983191d435c0d71c2bb1', '56aa371be4b08b9a8d573520', '4bf58dd8d48988d193941735', '4bf58dd8d48988d17b941735', '58daa1558bbb0b01f18ec1fd','50aaa49e4b90af0d42d5de11', '56aa371be4b08b9a8d573547', '4bf58dd8d48988d15a941735', '5744ccdfe4b0c0459246b4b5', '50aaa4314b90af0d42d5de10', '4bf58dd8d48988d161941735', '52e81612bcbc57f1066b7a21', '52e81612bcbc57f1066b7a13', '4bf58dd8d48988d162941735', '52e81612bcbc57f1066b7a14', '4bf58dd8d48988d163941735', '4bf58dd8d48988d164941735', '4bf58dd8d48988d165941735', '56aa371be4b08b9a8d573560', '58daa1558bbb0b01f18ec1d6', '52e81612bcbc57f1066b7a32', '4eb1daf44b900d56c88a4600', '4bf58dd8d48988d12d941735', '4bf58dd8d48988d12f941735', '52e81612bcbc57f1066b7a33', '4bf58dd8d48988d18d941735']

In [0]:
toronto_venues = toronto_venues[toronto_venues["Venue Category ID"].isin(list_of_catgories_id)]

## Analyzing Each Neighborhood

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Amphitheater,Aquarium,Arcade,Art Gallery,Art Museum,Baseball Stadium,Basketball Stadium,Castle,Circus,Comedy Club,Concert Hall,Dance Studio,Fountain,Gaming Cafe,Garden,General Entertainment,Historic Site,History Museum,Hockey Arena,Indie Movie Theater,Indie Theater,Jazz Club,Lake,Monument / Landmark,Movie Theater,Museum,Music Venue,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Plaza,Racetrack,Rock Club,Scenic Lookout,Soccer Stadium,Street Art,Theater,Theme Park,Zoo
4,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
12,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
13,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
29,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [12]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Amphitheater,Aquarium,Arcade,Art Gallery,Art Museum,Baseball Stadium,Basketball Stadium,Castle,Circus,Comedy Club,Concert Hall,Dance Studio,Fountain,Gaming Cafe,Garden,General Entertainment,Historic Site,History Museum,Hockey Arena,Indie Movie Theater,Indie Theater,Jazz Club,Lake,Monument / Landmark,Movie Theater,Museum,Music Venue,Opera House,Other Great Outdoors,Park,Performing Arts Venue,Plaza,Racetrack,Rock Club,Scenic Lookout,Soccer Stadium,Street Art,Theater,Theme Park,Zoo
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.076923,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.307692,0.076923,0.076923,0.000000,0.000000,0.000000,0.000000,0.000,0.076923,0.000000,0.000000
1,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,Central Bay Street,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.153846,0.000000,0.000,0.076923,0.000000,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.153846,0.000000,0.153846,0.000000,0.000000,0.000000,0.000000,0.000,0.153846,0.000000,0.000000
3,Christie,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.100000,0.0,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.100000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.100000,0.000000,0.000000
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.153846,0.000000,0.000,0.076923,0.076923,0.076923,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.384615,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.076923,0.000000,0.000000
5,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.571429,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
6,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.125000,0.125000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
7,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.142857,0.428571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
8,"Forest Hill West, Forest Hill North",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000,0.111111,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.444444,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.08333

A function to sort the venues in descending order.

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [14]:
num_top_venues = toronto_venues["Venue Category"].nunique()

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
0,Berczy Park,Park,Basketball Stadium,Museum,Theater,Fountain,Jazz Club,Lake,Plaza,Performing Arts Venue,Concert Hall,Gaming Cafe,Dance Studio,Comedy Club,Zoo,Circus,Castle,General Entertainment,Baseball Stadium,Art Museum,Art Gallery,Arcade,Aquarium,Garden,Indie Movie Theater,Historic Site,History Museum,Hockey Arena,Theme Park,Indie Theater,Monument / Landmark,Movie Theater,Music Venue,Opera House,Other Great Outdoors,Racetrack,Rock Club,Scenic Lookout,Soccer Stadium,Street Art,Amphitheater
1,Business Reply Mail Processing Centre 969 Eastern,Park,Indie Theater,Movie Theater,Comedy Club,General Entertainment,Garden,Gaming Cafe,Fountain,Dance Studio,Concert Hall,Zoo,History Museum,Circus,Castle,Basketball Stadium,Baseball Stadium,Art Museum,Art Gallery,Arcade,Aquarium,Historic Site,Indie Movie Theater,Hockey Arena,Theme Park,Theater,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Plaza,Performing Arts Venue,Other Great Outdoors,Opera House,Music Venue,Museum,Monument / Landmark,Lake,Jazz Club,Amphitheater
2,Central Bay Street,Theater,Plaza,Dance Studio,Park,Historic Site,Garden,Art Gallery,Concert Hall,Opera House,Comedy Club,General Entertainment,Gaming Cafe,Fountain,Zoo,Circus,History Museum,Basketball Stadium,Baseball Stadium,Art Museum,Arcade,Aquarium,Castle,Indie Movie Theater,Hockey Arena,Other Great Outdoors,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Performing Arts Venue,Music Venue,Theme Park,Museum,Movie Theater,Monument / Landmark,Lake,Jazz Club,Indie Theater,Amphitheater
3,Christie,Comedy Club,Indie Movie Theater,Castle,Museum,Music Venue,Park,Historic Site,Art Gallery,Theater,Baseball Stadium,Basketball Stadium,Art Museum,History Museum,Arcade,Aquarium,Concert Hall,Dance Studio,Fountain,Gaming Cafe,Garden,General Entertainment,Circus,Zoo,Hockey Arena,Performing Arts Venue,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Plaza,Other Great Outdoors,Theme Park,Opera House,Movie Theater,Monument / Landmark,Lake,Jazz Club,Indie Theater,Amphitheater
4,Church and Wellesley,Park,Dance Studio,Concert Hall,General Entertainment,Garden,Museum,Historic Site,Theater,Baseball Stadium,Basketball Stadium,Castle,History Museum,Circus,Comedy Club,Art Museum,Art Gallery,Arcade,Fountain,Gaming Cafe,Aquarium,Zoo,Indie Movie Theater,Hockey Arena,Theme Park,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Plaza,Performing Arts Venue,Other Great Outdoors,Opera House,Music Venue,Movie Theater,Monument / Landmark,Lake,Jazz Club,Indie Theater,Amphitheater


## Clustering Neighborhoods

In [15]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 2, 1, 1, 1, 1, 1, 0], dtype=int32)

In [16]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_tor
toronto_merged.columns = ['postal_code','borough',	'Neighborhood',	'lati',	'long']

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,postal_code,borough,Neighborhood,lati,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue,21th Most Common Venue,22th Most Common Venue,23th Most Common Venue,24th Most Common Venue,25th Most Common Venue,26th Most Common Venue,27th Most Common Venue,28th Most Common Venue,29th Most Common Venue,30th Most Common Venue,31th Most Common Venue,32th Most Common Venue,33th Most Common Venue,34th Most Common Venue,35th Most Common Venue,36th Most Common Venue,37th Most Common Venue,38th Most Common Venue,39th Most Common Venue,40th Most Common Venue
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1,Park,Theater,Performing Arts Venue,Historic Site,Circus,Rock Club,General Entertainment,Garden,Gaming Cafe,Fountain,Dance Studio,Concert Hall,Comedy Club,Zoo,Castle,History Museum,Baseball Stadium,Art Museum,Art Gallery,Arcade,Aquarium,Basketball Stadium,Indie Movie Theater,Hockey Arena,Opera House,Street Art,Soccer Stadium,Scenic Lookout,Racetrack,Plaza,Other Great Outdoors,Music Venue,Theme Park,Museum,Movie Theater,Monument / Landmark,Lake,Jazz Club,Indie Theater,Amphitheater
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,1,Park,Dance Studio,Plaza,General Entertainment,Museum,Concert Hall,Historic Site,Art Gallery,Basketball Stadium,Baseball Stadium,Castle,History Museum,Circus,Comedy Club,Art Museum,Arcade,Fountain,Gaming Cafe,Garden,Aquarium,Zoo,Indie Movie Theater,Hockey Arena,Theme Park,Theater,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Performing Arts Venue,Other Great Outdoors,Opera House,Music Venue,Movie Theater,Monument / Landmark,Lake,Jazz Club,Indie Theater,Amphitheater
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Theater,Plaza,Park,Music Venue,Garden,Dance Studio,Monument / Landmark,Concert Hall,Historic Site,Zoo,Gaming Cafe,Fountain,Comedy Club,Castle,Circus,Basketball Stadium,Baseball Stadium,Art Museum,Art Gallery,Arcade,Aquarium,General Entertainment,Indie Movie Theater,History Museum,Other Great Outdoors,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Performing Arts Venue,Opera House,Hockey Arena,Museum,Movie Theater,Lake,Jazz Club,Indie Theater,Theme Park,Amphitheater
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Theater,Plaza,Fountain,Jazz Club,Monument / Landmark,Park,Concert Hall,Museum,Zoo,Comedy Club,Garden,Gaming Cafe,Dance Studio,Castle,Circus,Historic Site,Basketball Stadium,Baseball Stadium,Art Museum,Art Gallery,Arcade,Aquarium,General Entertainment,Indie Movie Theater,History Museum,Hockey Arena,Theme Park,Indie Theater,Lake,Movie Theater,Music Venue,Opera House,Other Great Outdoors,Performing Arts Venue,Racetrack,Rock Club,Scenic Lookout,Soccer Stadium,Street Art,Amphitheater
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Park,Indie Movie Theater,Aquarium,Arcade,Historic Site,General Entertainment,Garden,Gaming Cafe,Fountain,Dance Studio,Concert Hall,Comedy Club,Hockey Arena,Circus,Castle,Basketball Stadium,Baseball Stadium,Art Museum,Art Gallery,History Museum,Zoo,Theme Park,Performing Arts Venue,Theater,Street Art,Soccer Stadium,Scenic Lookout,Rock Club,Racetrack,Plaza,Other Great Outdoors,Indie Theater,Opera House,Music Venue,Museum,Movie Theater,Monument / Landmark,Lake,Jazz Club,Amphitheater


In [17]:
address = 'Toronto, Ca'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Finally, let's visualize the resulting clusters**

In [21]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(0)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['lati'], toronto_merged['long'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters